# Transformer training (chantier en cours)

Imports

In [1]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

In [2]:
from statapp.common.preprocessing import load_data, encode_data, split_into_X_y

Preprocessing maison bien moche pour le moment... ^^ (essai avec un encodage sur les mots) Les données sont placées directement dans le dossier du notebook

In [3]:
text = load_data("data/fr.train.top1M.txt", sample=0.00001)

tokens = nltk.word_tokenize(text[:10])

In [4]:
len(tokens)

5

In [5]:
vocab = list(set(tokens))

vocab_size = len(vocab)

vocab_numbers = dict(zip(vocab, range(0,len(vocab))))
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

tokens_numbers_sequences = np.array([ tokens_numbers[i:i+max_length] for i in range(len(tokens_numbers)-max_length+1)])
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

nb_sequences =  tokens_numbers_sequences.shape[0]

In [6]:
vocab_size

5

In [7]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

#Correspond à utiliser l'entropie croisée puisque les sorties sont des log_softmax
#et l'entropie croisée = nll_loss(log_softmax(.), target)
criterion = nn.NLLLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.01)

In [8]:
print(tokens_numbers)

[2 3 4 1 0]


In [9]:
print(tokens_numbers_sequences)

tensor([[2, 3, 4],
        [3, 4, 1],
        [4, 1, 0]])


In [10]:
def train_model(nb_epochs, batch_size):
    
    #What is this ?? I don't remember
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for i, batch_indices in enumerate(batchs_indices):
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            print(output[:,:-1].reshape(-1, vocab_size))
            print(batch[:,1:].flatten())
            loss = criterion(output[:,:-1].reshape(-1, vocab_size), batch[:,1:].flatten())

            loss.backward()
            optimizer.step()
            
            #Il faudrait adapter les affichages en fonction du nombre de batchs total
            running_loss += loss.item()
            #if i % 3 == 3:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1))
            running_loss = 0.

Test d'overfitting sur un cas ultrasimplifié ;
- En observant les sorties le modèle a bien appris et overfitte, mais la loss affichée ne descend pas jusque zéro !

In [11]:
train_model(30,2)

C:\Users\Eric\statapp_language_model\statapp\transformer\pytorch\transformer_model.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.add(embedded, pos_encodings), dtype=torch.float32)


tensor([[-1.5251, -1.6545, -1.0780, -1.8589, -2.3528],
        [-1.8121, -1.7912, -0.8091, -1.6637, -3.3451],
        [-1.5907, -2.7622, -0.8608, -1.2930, -3.3299],
        [-1.6051, -1.7638, -1.0290, -1.8070, -2.2420]],
       grad_fn=<UnsafeViewBackward>)
tensor([4, 1, 3, 4])
[1,     1] loss: 1.920
tensor([[-2.7232, -1.9551, -2.8479, -2.9713, -0.3804],
        [-2.8264, -0.4072, -2.4277, -2.5519, -2.2154],
        [-2.6881, -0.4726, -2.2460, -2.1995, -2.3868],
        [-3.3086, -0.9947, -1.3371, -2.3686, -1.4382]],
       grad_fn=<UnsafeViewBackward>)
tensor([4, 1, 1, 0])
[2,     1] loss: 1.142
tensor([[-3.9297, -0.0769, -4.9547, -4.3922, -3.3535],
        [-1.7676, -1.2095, -2.3108, -2.4875, -1.0538],
        [-4.0554, -3.3070, -5.5248, -5.1662, -0.0658],
        [-3.9617, -0.1001, -5.0701, -4.7525, -2.7911]],
       grad_fn=<UnsafeViewBackward>)
tensor([1, 0, 4, 1])
[3,     1] loss: 0.503
tensor([[-4.7529, -0.0186, -7.4574, -6.4347, -4.8737],
        [-0.3356, -2.6706, -3.8856, -2.

       grad_fn=<UnsafeViewBackward>)
tensor([4, 1, 1, 0])
[23,     1] loss: 0.000
tensor([[-1.3562e+01, -1.3113e-06, -2.5690e+01, -2.3426e+01, -1.9633e+01],
        [-1.0729e-06, -2.5212e+01, -2.6714e+01, -1.3725e+01, -2.3532e+01],
        [-1.6440e+01, -1.9679e+01, -2.5266e+01, -2.5035e+01, -1.1921e-07],
        [-1.4310e+01, -5.9605e-07, -2.5938e+01, -2.4917e+01, -1.8300e+01]],
       grad_fn=<UnsafeViewBackward>)
tensor([1, 0, 4, 1])
[24,     1] loss: 0.000
tensor([[-1.6897e+01, -2.0011e+01, -2.5620e+01, -2.5490e+01,  0.0000e+00],
        [-1.4501e+01, -4.7684e-07, -2.6191e+01, -2.5169e+01, -1.8565e+01],
        [-1.3797e+01, -1.0729e-06, -2.5957e+01, -2.3727e+01, -1.9833e+01],
        [-1.0729e-06, -2.5463e+01, -2.6950e+01, -1.3777e+01, -2.3776e+01]],
       grad_fn=<UnsafeViewBackward>)
tensor([4, 1, 1, 0])
[25,     1] loss: 0.000
tensor([[-1.7324e+01, -2.0316e+01, -2.5946e+01, -2.5911e+01,  0.0000e+00],
        [-1.4679e+01, -4.7684e-07, -2.6420e+01, -2.5399e+01, -1.8805e+01],
  